LEZIONE 4
MATRICI TRIANGOLARI

1. 3.

In [217]:
using LinearAlgebra

function forward_substitution(L::AbstractMatrix, b::AbstractVector)       #risoluzione di un sistema lineare con matrice triangolare inferiore
    n = size(L, 1)
    if size(L, 2) != n || length(b) != n                        
        throw(ArgumentError("Matrix L must be square"))                   #controllo matrice quadrata
    end
    for i in 1:n 
        if L[i, i] == 0
            throw(ArgumentError("Matrix L is singular"))                  #controllo elemento diagonale di L diverso da zero
        end
    end
    x = similar(b, Float64)
    x[1] = b[1] / L[1, 1]
    for i in 2:n
        S =  0.0
        for j in 1:i-1
            S += L[i, j] * x[j]
        end
        x[i] = (b[i] - S)/ L[i, i]
    end

    return x
end

function backward_substitution(U::AbstractMatrix, b::AbstractVector)      #risoluzione di un sistema lineare con matrice triangolare superiore
    n = size(U, 1)
    if size(U, 2) != n || length(b) != n
        throw(ArgumentError("Matrix U must be square"))                   #controllo matrice quadrata
    end
    for i in 1:n
        if U[i, i] == 0                                                   #controllo elemento diagonale di U diverso da zero
            throw(ArgumentError("Matrix U is singular"))
        end
    end
    x = similar(b, Float64)
    x[n] = b[n] / U[n, n]
    for i in n-1:-1:1
        S = 0.0
        for j in i+1:n
            S += U[i, j] * x[j]
        end
        x[i] = (b[i] - S) / U[i, i]
    end

    return x
end

backward_substitution (generic function with 1 method)

2.

In [218]:
A_d = [-2 0 0;
      1 -1 0;
      3 2 1]
a_d = [-4; 2; 1]
x = forward_substitution(A_d, a_d)
println(x)
println("A * x = ", A_d * x)

B_d = [4 0 0 0;
     1 -2 0 0 ;
     -1 4 4 0 ;
     2 -5 5 1]
b_d = [-4; 1; -3; 5]
y = forward_substitution(B_d, b_d)
println(y)
println("B * y = ", B_d * y)

[2.0, -0.0, -5.0]
A * x = [-4.0, 2.0, 1.0]
[-1.0, -1.0, 0.0, 2.0]
B * y = [-4.0, 1.0, -3.0, 5.0]


4.

In [219]:
A_u = [3 1 0;
      0 -1 -2;
      0 0 3]
a_u = [1; 1; 6]
w = backward_substitution(A_u, a_u)
println(w)
println("A * w = ", A_u * w)

B_u = [3 1 0 6;
     0 -1 -2 7;
     0 0 3 4;
     0 0 0 5]
b_u = [4; 1; 1; 5]
z = backward_substitution(B_u, b_u)
println(z)
println("B * z = ", B_u * z)

[2.0, -5.0, 2.0]
A * w = [1.0, 1.0, 6.0]
[-3.3333333333333335, 8.0, -1.0, 1.0]
B * z = [4.0, 1.0, 1.0, 5.0]


5a.

In [220]:
function forward_substitution_matrix(L::AbstractMatrix, B::AbstractMatrix)       #risoluzione di p sistemi lineari con matrice triangolare inferiore
    n = size(B,1)
    p = size(B,2)
    if size(L,1) != n || size(L,2) != n 
        throw(ArgumentError("Matrix L must be square and compatible with B"))    #controllo matrice quadrata e compatibilità tra L e B
    end
    for i in 1:n 
        if L[i, i] == 0
            throw(ArgumentError("Matrix L is singular"))               #controllo elemento diagonale di L diverso da zero
        end
    end
    X = similar(B, Float64)
    for k in 1:p
        X[1, k] = B[1, k] / L[1, 1]
        for i in 2:n
            S = 0.0
            for j in 1:i-1
                S += L[i, j] * X[j, k]
            end
            X[i, k] = (B[i, k] - S) / L[i, i]
        end
    end
    return X
end

function backward_substitution_matrix(U::AbstractMatrix, B::AbstractMatrix)        #risoluzione di p sistemi lineari con matrice triangolare superiore
    n = size(B,1)
    p = size(B,2)
    if size(U, 1) != n || size(U,2) != n
        throw(ArgumentError("Matrix U must be square and compatible with B"))   #controllo matrice quadrata e compatibilità tra U e B
    end
    for i in 1:n
        if U[i, i] == 0
            throw(ArgumentError("Matrix U is singular"))                      #controllo elemento diagonale di U diverso da zero
        end
    end
    X = similar(B, Float64)
    for k in 1:p
        X[n, k] = B[n, k] / U[n, n]
        for i in n-1:-1:1
            S = 0.0
            for j in i+1:n
                S += U[i, j] * X[j, k]
            end
        X[i, k] = (B[i, k] - S) / U[i, i]
        end
    end
     return X
end

backward_substitution_matrix (generic function with 1 method)

5b. 

In [221]:
function inverse(A::AbstractMatrix)
    X = forward_substitution_matrix(A, I)
    return X
end

inverse (generic function with 1 method)

LEZIONE 5

1. 4.

In [222]:
using LinearAlgebra

function LU_decomposition(A::AbstractMatrix)
    n = size(A,1)

    if size(A,1) != n || size(A,2) != n                          #controllo matrice quadrata
        throw(ArgumentError("Matrix A must be square"))
    end
    
    L = diagm(ones(n))
    U = zeros(n,n);

    for i in 1:n
        U[i,:] = A[i,:]
    
        if U[i, i] == 0
                throw(ArgumentError("The algorithm fails"))       #controllo elemento diagonale di U
        end
        
        L[:,i] = A[:,i]/U[i,i]
        A = A - L[:,i]*U[i,:]'
    end
    
    return L, U
end
    


LU_decomposition (generic function with 1 method)

In [223]:
function LU_decomposition_eff(A::AbstractMatrix)                        #LU decomposition ottimizzata
    n = size(A,1)

    if size(A,1) != n || size(A,2) != n
        throw(ArgumentError("Matrix A must be square"))                 #controllo matrice quadrata
    end
    
    L = diagm(ones(n))
    U = zeros(n,n);
    
    for i in 1:n
        U[i,i:n] = A[i,i:n]  #ottimizzazione: non fa calcoli su zeri di U, che è una matrice triangolare superiore
    
        if U[i, i] == 0
                throw(ArgumentError("Matrix U is singular"))         #controllo elemento diagonale di U
        end
        
        L[i+1:n,i] = A[i+1:n,i]/U[i,i]
        A = A - L[:,i]*U[i,:]'
    end
    
    return L, U
end

function determinante(A::AbstractMatrix)
    L, U = LU_decomposition_eff(A)
    return prod(diag(U))
end

determinante (generic function with 1 method)

2. a.

In [224]:
A = [
     2    3    4     
     4    5    10   
     4    8    2    
    ];     

L, U = LU_decomposition(A)                                  #LU decomposition non ottimizzata
L_2, U_2 = LU_decomposition_eff(A)
println("L_2 = ", L_2)
println("L = ", L)
println("U = ", U)
println("L*U = ", L*U)
println("det_A = ", determinante(A))

L_2 = [1.0 0.0 0.0; 2.0 1.0 0.0; 2.0 -2.0 1.0]
L = [1.0 -0.0 -0.0; 2.0 1.0 -0.0; 2.0 -2.0 1.0]
U = [2.0 3.0 4.0; 0.0 -1.0 2.0; 0.0 0.0 -2.0]
L*U = [2.0 3.0 4.0; 4.0 5.0 10.0; 4.0 8.0 2.0]
det_A = 4.0


2. b.

In [225]:
B = [
     6    -2    -4     4
     3    -3    -6     1
     -12   8     21    -8
     -6    0    -10    7
    ];     

L, U = LU_decomposition(B)
println("L = ", L)
println("U = ", U)
println("L*U = ", L*U)
println("det_B = ", determinante(B))


L = [1.0 -0.0 0.0 0.0; 0.5 1.0 0.0 0.0; -2.0 -2.0 1.0 0.0; -1.0 1.0 -2.0 1.0]
U = [6.0 -2.0 -4.0 4.0; 0.0 -2.0 -4.0 -1.0; 0.0 0.0 5.0 -2.0; 0.0 0.0 0.0 8.0]
L*U = [6.0 -2.0 -4.0 4.0; 3.0 -3.0 -6.0 1.0; -12.0 8.0 21.0 -8.0; -6.0 0.0 -10.0 7.0]
det_B = -480.0


2. c.

In [226]:
C = [
     1    4    5    -5
    -1    0   -1    -5
     1    3   -1     2
     1   -1    5    -1
    ];     

L, U = LU_decomposition(C)
println("L = ", L)
println("U = ", U)
println("L*U = ", L*U)
println("det_C = ", determinante(C))


L = [1.0 0.0 -0.0 -0.0; -1.0 1.0 -0.0 -0.0; 1.0 -0.25 1.0 -0.0; 1.0 -1.25 -1.0 1.0]
U = [1.0 4.0 5.0 -5.0; 0.0 4.0 4.0 -10.0; 0.0 0.0 -5.0 4.5; 0.0 0.0 0.0 -4.0]
L*U = [1.0 4.0 5.0 -5.0; -1.0 0.0 -1.0 -5.0; 1.0 3.0 -1.0 2.0; 1.0 -1.0 5.0 -1.0]
det_C = 80.0


In [227]:
3. 

3.0

In [228]:
function T(x, y)
    T = [
        1    0    x     
        0    1    y   
        0    0    1    
        ];  
    return T
end

function R(theta)
    R = [
        cos(theta)    sin(theta)    0     
        -sin(theta)    cos(theta)    0   
            0              0         1    
        ];  
    return R
end
A_3 = zeros(3,3)
A_3 = T(3, -1) * R(pi/5) * T(-3, 1)

z = [2; 2; 1]

3-element Vector{Int64}:
 2
 2
 1

3. a.

In [229]:
b = A_3 * z
println("b = ", b)

b = [3.9543387625024717, 2.0148362354173153, 1.0]


3. b.

In [230]:
L, U = LU_decomposition(A_3)
println("L = ", L)
println("U = ", U)
println("L*U = ", L*U)

L = [1.0 0.0 0.0; -0.7265425280053609 1.0 0.0; 0.0 0.0 1.0]
U = [0.8090169943749475 0.5877852522924731 1.1607342691676308; 0.0 1.2360679774997898 2.4156955615158724; 0.0 0.0 1.0]
L*U = [0.8090169943749475 0.5877852522924731 1.1607342691676308; -0.5877852522924731 0.8090169943749475 1.5723727512523669; 0.0 0.0 1.0]


3. c.

In [231]:
function LU_solution(A::AbstractMatrix, b::AbstractVector)          #soluzione di un sistema lineare con LU decomposition
    L, U = LU_decomposition_eff(A)
    z = forward_substitution(L, b)
    x = backward_substitution(U, z)
    return x
end

return x 
x = LU_solution(A_3, b)
print(x-z)

[0.0, -6.661338147750939e-16, 0.0]

4. 

In [232]:
using LinearAlgebra

function row_pivoting(A::AbstractMatrix)
    n = size(A,1)

    if size(A,1) != n || size(A,2) != n                         #controllo matrice quadrata
        throw(ArgumentError("Matrix A must be square"))
    end

    L_1 = diagm(ones(n))
    U = zeros(n,n);
    index = collect(1:n)
    P = zeros(n,n)
    for i in 1:n
        j = argmax( abs.(A[:,i]) ) 
        index[i] = j
        P[i,j] = 1
        U[i,i:n] = A[j,i:n]    #ottimizzazione: non fa calcoli su zeri di U, che è una matrice triangolare superiore
    
        if U[i, i] == 0
                throw(ArgumentError("Matrix A is singular"))       #controllo elemento diagonale di U
        end
        
        L_1[:,i] = A[:,i]/U[i,i]
        A = A - L_1[:,i]*U[i,:]'
    end
    L = P*L_1
    return L, U, P
end

function det_permutation(P::AbstractMatrix)              #calcola il determinante della matrice di permutazione 
    n = size(P,1)
    p = zeros(n)
    for i in 1:n
        for j in 1:n
            if P[i,j] == 1
                p[i] = j
            end
        end
    end
    
    S = 0 
    for i in 1:n
        while p[i] != i
            p[p[i]], p[i] = p[i], p[p[i]]   #scambio per contare le inversioni
            S += 1
        end
    end
    return (-1)^S
end
        
function determinante_rp(A::AbstractMatrix)             #determinante con row pivoting
    n = size(A,1)
    _, U, P = row_pivoting(A)
    det_P = det_permutation_Imatrix(P)
    det_A = det_P * prod(diag(U))
    return det_A
end

function row_pivoting_solution(A::AbstractMatrix, b::AbstractVector)       #risoluzione di un sistema lineare con row pivoting
    L, U, P = row_pivoting(A)
    z = forward_substitution(L, P*b)
    x = backward_substitution(U, z)
    return x
end




row_pivoting_solution (generic function with 1 method)

In [233]:

L, U, P = row_pivoting(A)

println("L = ", L)
println("U = ", U)
println("P = ", P)
println(P*A-L*U)
println("det_A = ", determinante_rp(A))



L = [1.0 0.0 0.0; 1.0 1.0 0.0; 0.5 0.16666666666666666 1.0]
U = [4.0 5.0 10.0; 0.0 3.0 -8.0; 0.0 0.0 0.33333333333333326]
P = [0.0 1.0 0.0; 0.0 0.0 1.0; 1.0 0.0 0.0]
[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
det_A = 3.999999999999999


In [234]:
L, U, P = row_pivoting(B)

println("L = ", L)
println("U = ", U)
println("P = ", P)
println(P*B-L*U)
println("det_B = ", determinante_rp(B))

L = [1.0 0.0 0.0 0.0; 0.5 1.0 0.0 0.0; -0.25 0.25 1.0 0.0; -0.5 -0.5 -0.8571428571428571 1.0]
U = [-12.0 8.0 21.0 -8.0; 0.0 -4.0 -20.5 11.0; 0.0 0.0 4.375 -3.75; 0.0 0.0 0.0 2.285714285714286]
P = [0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 1.0 0.0 0.0; 1.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
det_B = -480.00000000000006


In [235]:
L, U, P = row_pivoting(C)

println("L = ", L)
println("U = ", U)
println("P = ", P)
println(P*C-L*U)
println("det_C = ", determinante_rp(C))

L = [1.0 0.0 0.0 0.0; 1.0 1.0 0.0 0.0; 1.0 0.2 1.0 0.0; -1.0 -0.8 -0.6666666666666666 1.0]
U = [1.0 4.0 5.0 -5.0; 0.0 -5.0 0.0 4.0; 0.0 0.0 -6.0 6.2; 0.0 0.0 0.0 -2.666666666666667]
P = [1.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 1.0 0.0; 0.0 1.0 0.0 0.0]
[0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 -2.220446049250313e-16 2.220446049250313e-16 0.0]
det_C = 80.00000000000001


In [236]:
x = row_pivoting_solution(A_3, b)
print(x-z)

[0.0, -6.661338147750939e-16, 0.0]

Conditioning of linear systems

Esercizio 1

In [237]:
ε₁ = 1e-12

D = [
    -ε₁    1
     1   -1
    ]

2×2 Matrix{Float64}:
 -1.0e-12   1.0
  1.0      -1.0

1. a.

In [238]:

x = [1; 1]
d = D * x

2-element Vector{Float64}:
 0.999999999999
 0.0

1. b.

In [239]:
L₁, U₁ = LU_decomposition_eff(D)
println("L = ", L₁)
println("U = ", U₁)
x_sol1 = LU_solution(D, d)
t₁ = 0
if x-x_sol1 != 0
    t₁ = -log10(norm((x-x_sol1)/norm(x)))
    println("Il numero di cifre accurate è ", t₁)
else
    println("La soluzione è esatta")
end

L = [1.0 0.0; -1.0e12 1.0]
U = [-1.0e-12 1.0; 0.0 9.99999999999e11]
Il numero di cifre accurate è 4.805696104393144


1. c.

In [240]:
D - L₁ * U₁


2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

1. d.

In [241]:

ε₂ = 1e-20

E = [
    -ε₂    1
     1   -1
    ]
   
x = [1; 1]
e = E * x

L₂, U₂ = LU_decomposition_eff(E)
println("L = ", L₂)
println("U = ", U₂)
x_sol2 = LU_solution(E, e)
t₂ = 0
if x-x_sol2 != 0
    t₂ = -log10(norm((x-x_sol2)/norm(x)))
    println("Il numero di cifre accurate è ", t₂)
else
    println("La soluzione è esatta")
end

E - L₂ * U₂


L = [1.0 0.0; -1.0e20 1.0]
U = [-1.0e-20 1.0; 0.0 1.0e20]
Il numero di cifre accurate è 0.15051499783199063


2×2 Matrix{Float64}:
 0.0   0.0
 0.0  -1.0

In [242]:
function inverse_2x2(A::AbstractMatrix)
    if size(A,1) != 2 || size(A,2) != 2
        throw(ArgumentError("Matrix A must be 2x2"))
    end
    det_A =A[1,1]*A[2,2] - A[1,2]*A[2,1]
    if det_A == 0
        throw(ArgumentError("Matrix A can't be inverted"))
    end
    return 1/det_A * [A[2,2] -A[1,2]; -A[2,1] A[1,1]]
end

inverse_2x2 (generic function with 1 method)

In [ ]:
#scelgo norma 1 erchè computazionalmente più efficiente in julia, in cui si percorrono in memoria le colonne: fisso la colonna e sommo gli elementi di ogni riga

function norma_1(A::AbstractMatrix)
    n = size(A,1)
    m = size(A,2)
    somme = zeros(m)

    for j in 1:m
        s = 0.0
        for i in 1:n
            s += abs(A[i,j])
        end
    somme[j] = s
    end
    return maximum(somme)
end

println("Il numero di cifre accurate rispettivamente della prima e della seconda matrice sono:")
k_D = norma_1(D) * norma_1(inverse_2x2(D))
println("k(D) = ", k_D)
t_1 = log10(k_D)
println("t_1 = ", t_1)


k_E = norma_1(E) * norma_1(inverse_2x2(E))
println("k(E) = ", k_E)
t_2 = log10(k_E)
println("t_2 = ", t_2)

Il numero di cifre accurate per il primo e il secondo sistema sono:
k(D) = 4.000000000004
t_1 = 0.6020599913283967
k(E) = 4.0
t_2 = 0.6020599913279624


Ax = b è ill-conditioned